<a href="https://colab.research.google.com/github/crisprking/miniprotein_genai/blob/main/notebooks/04_pipeline_end_to_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎛️ End‑to‑end pipeline (RFdiffusion → MP‑NN → AF2)
seq_len   = 100   #@param {type:"integer"}
run_name  = "demo" #@param {type:"string"}
num_vars  = 4      #@param {type:"integer"}

# 1. Backbone
import subprocess, pathlib, os, sys
rf_cmd = [
    "python","-m","RFdiffusion.run_inference",
    f"contigmap.contigs=[{seq_len}]",
    f"inference.output_prefix=outputs/{run_name}",
    "diffuser.T=50","inference.num_designs=1",
    "inference.dump_pdb=True","inference.dump_pdb_path=/dev/shm"
]
subprocess.check_call(rf_cmd)
pdb = f"outputs/{run_name}_0.pdb"

# 2. ProteinMPNN
mpnn_cmd = [
    "python","-m","proteinmpnn.run",
    "--pdb_path", pdb,"--num_seq_per_target", str(num_vars),
    "--sampling_temp","0.1","--out_folder",f"outputs/{run_name}_mpnn"
]
subprocess.check_call(mpnn_cmd)
fa = sorted(pathlib.Path(f"outputs/{run_name}_mpnn").glob("*.fa"))[0]

# 3. ColabFold
from colabfold.batch import run, get_queries
q,_ = get_queries(str(fa))
run(q, result_dir=f"outputs/{run_name}_af2", model_type="auto", use_templates=False)
print("🎉 Pipeline finished → outputs/")

